# Статистика, DS-поток
## Практическое задание 6


**Правила:**

* Дедлайн **19 октября 16:00**. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Выполненную работу нужно отправить телеграм-боту `@miptstats_ad21_bot`.
* Прислать нужно ноутбук в формате `ipynb`.
* Решения, размещенные на каких-либо интернет-ресурсах не принимаются. Публикация решения может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качествие основы, ничего не удаляя из него.
* Никакой код из данного задания при проверке запускаться не будет.

**Баллы за задание:**

* Задача 1 &mdash; 5 баллов
* Задача 2 &mdash; 5 баллов
* Задача 3 &mdash; 5 баллов
* Задача 4a &mdash; 5 баллов
* Задача 4b &mdash; 10 баллов

**Важность задач:**

* *высокая:* задачи 1, 2, 3;
* *средне-высокая:* задача 4.

*Замечания.*
1. Вы должны помнить о разнице между доверительным интервалом и *реализацией* доверительного интервала. На практике обычно слово *реализация* опускается.
2. Если интервал получен на лекции/семинаре, то нужно просто выписать его формулу. Если такой случай не рассматривался, то нужно добавить его вывод.
3. Выборку надо сгенерировать один раз. Дело в том, что на практике при недостаточном размере выборки разумнее дособрать выборку, чем заново проводить все измерения.

In [445]:
import scipy.stats as sps
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.offline as pof
np.set_printoptions(precision=5)

______________
## Задача 1.

В этой задаче нужно визуализировать доверительные интервалы для выборки из равномерного распределения.

Чтобы не плодить код, напишите следующую функцию (см. ниже). При выборе стиля графика помните, что если изображаются лишь точки и линии, то лучше использовать серый фон, а если присутствуют закрашенные области, то предпочтительнее белый. Графики первого типа еще называют "легкими", а второго — "тяжелыми".

In [483]:
def draw_confidence_interval(
    left, right, estimation=None, sample=None, opacity=None,
    ylim=(None, None), color_estimation='#FF3300', color_interval='#00CC66', color_left='#00FFFF', color_right='#FF44CC',
    color_sample='#0066FF', label_estimation='Оценка', label_interval='Доверительный интервал', i=0
):
    '''
    Рисует доверительный интервал и оценку в зависимости от размера выборки.
    :param left: левые границы интервалов (в зависимости от n)
    :param right: правые границы интервалов (в зависимости от n)
    :param estimation: оценки (в зависимости от n)
    :param sample: выборка
    :param opacity: прозрачность закрашиваемого интервала
    :param ylim: ограничение вертикальной оси
    :param color_estimation: цвет оценки
    :param color_interval: цвет интервала
    :param color_left: цвет левой границы интервала
    :param color_right: цвет правой границы интервала
    :param color_sample: цвет выборки
    :param label_estimation: подпись для оценки
    :param label_interval: подпись для интервала
    :param i: номер для сохранения графика в html
    '''
    N = len(sample)
    t = np.linspace(1, N, N)
    t_rev = np.flip(t)
    fig = go.Figure()
    if len(left.shape) > 1:
        for i in range(left.shape[0]):
            right_rev = np.flip(right[i])
            fig.add_trace(go.Scatter(x=np.concatenate([t, t_rev]), y=np.concatenate([left[i], right_rev]), opacity=opacity,
                             fill='toself', fillcolor=color_interval[i], name=label_interval[i]))
            fig.add_trace(go.Scatter(x=t, y=left[i], line_color=color_left[i], 
                                     name=f'Левая граница интервала {label_interval[i]}'))
            fig.add_trace(go.Scatter(x=t, y=right[i], line_color=color_right[i], 
                                     name=f'Правая граница интервала {label_interval[i]}'))
            if estimation is not None:
                fig.add_trace(go.Scatter(x=t, y=estimation[i], line_color=color_estimation[i],
                                 name=label_estimation[i]))
    else:
        right_rev = np.flip(right)        
        fig.add_trace(go.Scatter(x=np.concatenate([t, t_rev]), y=np.concatenate([left, right_rev]), opacity=opacity,
                                 fill='toself', fillcolor=color_interval, name=label_interval))
        fig.add_trace(go.Scatter(x=t, y=left, line_color=color_left, name='Левая граница интервала'))
        fig.add_trace(go.Scatter(x=t, y=right, line_color=color_right, name='Правая граница интервала'))
        if estimation is not None:
            fig.add_trace(go.Scatter(x=t, y=estimation, line_color=color_estimation,
                                 name=label_estimation))
    fig.add_trace(go.Scatter(x=t, y=sample, line_color=color_sample, mode='lines+markers',
                                 name='Выборка'))
    fig.update_yaxes(range=ylim)  

    fig.update_layout(title='Доверительный интервал, оценка и выборка в зависимости от размера выборки',
                      xaxis_title='$n$', yaxis_title='$y$', plot_bgcolor = "white")
    fig.show()
    pof.plot(fig, filename=f'{i}.html', auto_open=False)

Пусть $X = (X_1, ..., X_n)$ &mdash; выборка из распределения $U[0, \theta]$. Постройте асимптотические доверительные интервалы: Вальда и на основе аппроксимации $X_{(n)}$ некоторым распределением. Для этого сгенерируйте выборку $X_1, ... X_{N}, N = 100$ и постройте график доверительных интервалов уровня доверия $0.95$, вычисленных для всех подвыборок размера $n$ вида $X_1, ... X_n$, $1 \le n \le 100$, используя написанную выше функцию. Нужно нанести на график точки выборки.

Для вычисления квантилей у каждого распределения из `scipy.stats` используйте функцию `ppf`.

Для двух статистик, используемых при построении интервалов, запишите аппроксимацию некоторым распределением неформально в виде "статистика по выборке достаточно большого размера приближенно имеет такое-то распределение". 

Сделайте вывод о том, как влияет на ширину интервала вид статистики и аппроксимация.

**Решение:**

In [484]:
theta = 1
N = 100
alpha = 0.95
z = sps.norm.ppf((1 + alpha) / 2)
u = sps.expon.ppf(alpha)

cumavg = lambda X: np.cumsum(X) / np.linspace(1, len(X), len(X))
cummax = lambda X: pd.Series(X.tolist()).cummax().to_numpy()
sqrtn = lambda N: np.sqrt(np.linspace(1, N, N))

X = sps.uniform(0, theta).rvs(size=N)
cum_avg = cumavg(X)
cum_max = cummax(X)
sqrt_n = sqrtn(N)
wald_interval_left = 2 * cum_avg - (z * 2 * cum_avg / (3 ** 0.5) / sqrt_n)
wald_interval_right = 2 * cum_avg + (z * 2 * cum_avg / (3 ** 0.5) / sqrt_n)
max_interval_left = cum_max
max_interval_right = cum_max / (1 - u / np.linspace(1, N, N))
avg_est = 2 * cum_avg
max_est = cum_max


draw_confidence_interval(wald_interval_left, wald_interval_right, avg_est, X, 
                         label_estimation='Оценка через среднее', label_interval='Асимптотический интервал Вальда', i=0)
draw_confidence_interval(max_interval_left, max_interval_right, max_est, X, ylim=(0.9, 1.1),
                         label_estimation='Оценка через максимум', 
                         label_interval='Асимптотический интервал через максимум', i=1)

**Вывод:** среднее примерно аппроксимируется распределением $\mathcal{N}(\frac{\theta}{2}, \frac{\theta}{\sqrt{3n}}),$ а максимум примерно аппроксимируется распределением $-Exp(\frac{1}{\theta n}).$ То есть, сходимость к искомому параметру у оценки $2\overline{X}$ &mdash; как корень из $n$, а у максимума &mdash; как $n$. Это же мы видим на графиках: длина доверительного интервала Вальда уменьшается примерно как $\sqrt{n}$ и на последнем шаге составляет около $0.3$, а доверительный интервал, построенный на основе максимума, имеет длину около $0.03$ на последнем шаге, которая убывает примерно пропорционально $n$.
Также заметим, что при малых $n$ доверительный интервал, построенный на основе максимума, имеет вести себя странно (особенно при $n \leq 3$, так как знаменатель правой границы при $n = 1, 2$ отрицательный, а при $n = 3$ положительный, но очень близкий к нулю, так как $0.95$-квантиль экспоненциального распределения с параметром 1 очень близок к 3).

__________________
### Задача 2.
Аналогично задаче 1 сгенерируйте выборку $X_1, ... X_{100}$ из стандартного распределения Коши и постройте доверительные интервалы для следующих случаев:

* точный доверительный интервал минимальной длины в параметрической модели $\mathcal{N}(\theta, 1)$ (см. замечание ниже);
* точный асимптотический доверительный интервал в параметрической модели распределения Коши со сдвигом, используя выборочную медиану;
* точный асимптотический доверительный интервал в параметрической модели распределения Коши со сдвигом, используя асимптотически эффективную оценку.

Изобразите интервалы *на одном* графике полупрозрачными цветами. Точки выборки нужно нанести на график. 

*Замечание:*

1. Первый пример призван проиллюстрировать, что бывает, если используется неправильная модель. На практике вы никогда не знаете, из какого семейства распределений на самом деле получены данные.
**Решение:**

In [485]:
theta = 1
X = sps.cauchy(theta, 1).rvs(size=N)
sigma = 1

cummedian = lambda X: pd.Series(X.tolist()).expanding().apply(np.median, raw=True).to_numpy()
def theta_asymptotically_efficient(X, theta_0):
    theta_opt = np.array([])
    for i in range(len(X)):
        X_minus_theta_0 = X[:i+1] - theta_0[i]
        a1 = X_minus_theta_0
        a2 = 1 + X_minus_theta_0 ** 2
        a3 = 1 - X_minus_theta_0 ** 2
        a4 = a2 ** 2
        cur_theta_opt = theta_0[i] + (np.sum(a1 / a2) / np.sum(a3 / a4))
        theta_opt = np.append(theta_opt, cur_theta_opt)
    return theta_opt

cum_avg = cumavg(X)
sqrt_n = sqrtn(N)

norm_left = cum_avg - (z * sigma / sqrt_n)
norm_right = cum_avg + (z * sigma / sqrt_n)

cum_median = cummedian(X)
sigma_median = np.pi / 2

median_left = cum_median - (z * sigma_median / sqrt_n)
median_right = cum_median + (z * sigma_median / sqrt_n)

theta_eff = theta_asymptotically_efficient(X, cum_median)
sigma_eff = 2 ** 0.5

eff_left = theta_eff - (z * sigma_eff / sqrt_n)
eff_right = theta_eff + (z * sigma_eff / sqrt_n)

left = np.array([norm_left, median_left, eff_left])
right = np.array([norm_right, median_right, eff_right])
est = np.array([cum_avg, cum_median, theta_eff])
labels_est = np.array(['Оценка через среднее', 'Оценка через медиану', 'Оценка через эффективную оценку'])
labels_int = np.array(['Среднее', 'Медиана', 'Эффективная оценка'])
colors_est = np.array(['#440000', '#884400', '#CC8844'])
colors_int = np.array(['#0044FF', '#FF0000', '#00FF44'])
colors_left = np.array(['#448888', '#88CC88', '#CCFFCC'])
colors_right = np.array(['#448800', '#88CC00', '#CCFF44'])



draw_confidence_interval(left=left, right=right, estimation=est, sample=X, label_estimation=labels_est, opacity=0.2,
                         label_interval=labels_int, color_estimation=colors_est, color_interval=colors_int, 
                         color_left=colors_left, color_right=colors_right, i=2)

**Вывод:** как и ожидалось, даже самая лучшая оценка для нормального распределения не работает для распределения Коши, поэтому, скорее всего, мы сможем понять на практике, что мы берем оценку для неправильного распределения. Также стоит отметить, что у асимптотически эффективной оценки ожидаемо меньше длина интервала, при этом она чуть ближе к искомому параметру.

__________________
## Задача 3.
 В данном задании вам нужно изучить доверительные интервалы для параметра сдвига в нормальной модели в случае неизвестной дисперсии. Требуется построить: 
 * асимтотический доверительный интервал при помощи центральной предельной теоремы и леммы Слуцкого;
 * точный неасимптотический при помощи распределений хи-квадрат, Стьюдента.

Вывод этих интервалов был разобран на лекции. Выпишите только ответы.

Асимптотический доверительный интервал: <...>

Точный доверительный интервал: <...>

Реализуйте функции построения этих интервалов по выборке. Задокументируйте функции (см. [гайд](https://realpython.com/documenting-python-code/)).

In [470]:
t = sps.t(N - 1).ppf((1 + alpha) / 2)
t

1.9842169515086827

In [471]:
cumvar = lambda X: pd.Series(X.tolist()).expanding().apply(np.var, raw=True).to_numpy()

def calculate_asymptotic_confidence_intervals(sample, alpha=0.95):
    '''
    Считает асимптотический доверительный интервал уровня доверия 'alpha' по выборке 'sample' 
    с помощью ЦПТ и леммы Слуцкого
    :param sample: выборка
    :param alpha: уровень доверия
    Вовзращает массивы левых и правых границ интервала
    '''
    N = len(sample)
    avg = cumavg(sample)
    S = cumvar(sample)
    left = avg - (z * S / sqrtn(N))
    right = avg + (z * S / sqrtn(N))
    return left, right

def calculate_confidence_intervals(sample, alpha=0.95):
    '''
    Считает точный доверительный интервал уровня доверия 'alpha' по выборке 'sample' 
    с помощью распределений хи-квадрат и Стьюдента
    :param sample: выборка
    :param alpha: уровень доверия
    Вовзращает массивы левых и правых границ интервала
    '''
    N = len(sample)
    avg = cumavg(sample)
    S = cumvar(sample)
    sqrt_1 = np.concatenate([[0.01], sqrtn(N - 1)])
    left = avg - (t * S / sqrt_1)
    right = avg + (t * S / sqrt_1)
    return left, right

Сгенерируйте выборку из нормального расределения и сравните два доверительных интервала в зависимости от размера выборки. Для сравнения отобразите оба интервала на одном графике. Проследите за тем, чтобы было видно, как соотносятся размеры интервалов. Поясните теоретическую причину такого поведения доверительных интервалов.

*Указание*: рассматривайте длину выборки около 20-30.

In [486]:
N = 30
X = sps.norm.rvs(size=N)
left_asymptotic, right_asymptotic = calculate_asymptotic_confidence_intervals(X)
left_exact, right_exact = calculate_confidence_intervals(X)

draw_confidence_interval(i=42, left=np.array([left_asymptotic, left_exact]), 
                         right=np.array([right_asymptotic, right_exact]), sample=X, ylim=(-0.5, 0.5),
                         label_interval=np.array(['Асимптотический интервал', 'Точный интервал']), 
                         color_interval=np.array(['#FF0000', '#00FF44']), opacity=0.3,
                         color_left=np.array(['#88CC88', '#CCFFCC']), color_right=np.array(['#88CC00', '#CCFF44']))

**Вывод:** начиная с $n = 2$, асимптотический и точный интервалы почти совпали, при этом незначительно больше длину имеет точный интервал. С точки зрения теории они так близки, потому что распределение Стьюдента стремится к стандартному нормальному с ростом $n$, значит, и квантили распреления тоже стремится к квантили стандартного нормального, также корни из $n$ и из $n - 1$ отличаются незначительно.

Cкачайте данные <a href="http://archive.ics.uci.edu/ml/datasets/wine">`wine dataset`</a> и рассмотрите столбцы `Alcalinity of ash`, `Nonflavanoid phenols`, `Proanthocyanins` и `Hue` для вина *первого типа*. Тип вина указан в первом столбце.

Постройте доверительные интервалы для параметров сдвига каждого из столбцов, предполагая, что столбцы имеют нормальное распределение. Нужно построить доверительные интервалы обоих рассмотренных выше типов. Запишите их в виде таблицы.

In [158]:
data = pd.read_csv('wine.data')
data_first = data.columns.values
data = data.rename(columns={data_first[0]: 'Sort', data_first[1]: 'Alcohol',
                            data_first[2]: 'Malic acid', data_first[3]: 'Ash',
                            data_first[4]: 'Alcalinity of ash', data_first[5]: 'Magnesium',
                            data_first[6]: 'Total phenols', data_first[7]: 'Flavanoids',
                            data_first[8]: 'Nonflavanoid phenols', data_first[9]: 'Proanthocyanins',
                            data_first[10]: 'Color intensity', data_first[11]: 'Hue',
                            data_first[12]: 'OD280/OD315 of diluted wines', data_first[13]: 'Proline'})
data_norm = data[["Alcalinity of ash", "Nonflavanoid phenols", "Proanthocyanins", "Hue"]]

left_alc_asympt, right_alc_asympt = calculate_asymptotic_confidence_intervals(sample=data["Alcalinity of ash"].to_numpy())
left_alc_exact, right_alc_exact = calculate_confidence_intervals(sample=data["Alcalinity of ash"].to_numpy())
left_mag_asympt, right_mag_asympt = calculate_asymptotic_confidence_intervals(sample=data["Magnesium"].to_numpy())
left_mag_exact, right_mag_exact = calculate_confidence_intervals(sample=data["Magnesium"].to_numpy())
left_pro_asympt, right_pro_asympt = calculate_asymptotic_confidence_intervals(sample=data["Proanthocyanins"].to_numpy())
left_pro_exact, right_pro_exact = calculate_confidence_intervals(sample=data["Proanthocyanins"].to_numpy())
left_hue_asympt, right_hue_asympt = calculate_asymptotic_confidence_intervals(sample=data["Hue"].to_numpy())
left_hue_exact, right_hue_exact = calculate_confidence_intervals(sample=data["Hue"].to_numpy())
d = {'left_alc_asympt' : left_alc_asympt, 'right_alc_asympt' : right_alc_asympt, 
     'left_alc_exact' : left_alc_exact, 'right_alc_exact' : right_alc_exact,
     'left_mag_asympt' : left_mag_asympt, 'right_mag_asympt' : right_mag_asympt, 
     'left_mag_exact' : left_mag_exact, 'right_mag_exact' : right_mag_exact,
     'left_pro_asympt' : left_pro_asympt, 'right_pro_asympt' : right_pro_asympt, 
     'left_pro_exact' : left_pro_exact, 'right_pro_exact' : right_pro_exact,
     'left_hue_asympt' : left_hue_asympt, 'right_hue_asympt' : right_hue_asympt, 
     'left_hue_exact' : left_hue_exact, 'right_hue_exact' : right_hue_exact}
data_int = pd.DataFrame(d)
data_int

,left_alc_asympt,right_alc_asympt,left_alc_exact,right_alc_exact,left_mag_asympt,right_mag_asympt,left_mag_exact,right_mag_exact,left_pro_asympt,right_pro_asympt,left_pro_exact,right_pro_exact,left_hue_asympt,right_hue_asympt,left_hue_exact,right_hue_exact
0,11.200000,11.200000,11.200000,11.200000,100.000000,100.000000,100.000000,100.000000,1.280000,1.280000,1.280000,1.280000,1.050000,1.050000,1.050000,1.050000
1,-4.073023,33.873023,-12.263930,42.063930,100.153524,100.846476,100.003946,100.996054,1.233934,2.856066,0.883787,3.206213,1.039861,1.040139,1.039802,1.040198
2,4.297944,26.768722,1.602573,29.464093,65.186898,144.146436,55.715697,153.617636,1.643929,2.536071,1.536916,2.643084,0.971777,0.988223,0.969804,0.990196
3,4.111235,29.688765,1.950074,31.849926,49.691071,166.308929,39.837503,176.162497,1.719373,2.325627,1.668148,2.376852,0.988998,1.001002,0.987983,1.002017
4,7.003801,26.116199,5.743637,27.376363,64.833629,152.766371,59.035839,158.564161,1.794714,2.229286,1.766060,2.257940,1.001281,1.010719,1.000658,1.011342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,17.807927,21.116928,17.782592,21.142263,69.643768,129.072995,69.188752,129.528011,1.542064,1.640826,1.541308,1.641582,0.957095,0.972223,0.956979,0.972339
173,17.831875,21.133642,17.806624,21.158894,69.908817,128.838310,69.458127,129.289000,1.541419,1.639386,1.540669,1.640135,0.955579,0.970696,0.955464,0.970812
174,17.848736,21.122692,17.823724,21.147704,69.920738,129.062119,69.468923,129.513934,1.540416,1.637642,1.539673,1.638384,0.953395,0.968617,0.953278,0.968733
175,17.865369,21.111904,17.840594,21.136679,69.937873,129.278036,69.485032,129.730877,1.540082,1.636509,1.539346,1.637244,0.951300,0.966610,0.951183,0.966726


In [372]:
def mle_for_mean(sample):
    '''
    :param sample: выборка из многомерного нормального распределения
    :return: ОМП для вектора средних
    '''
    
    return np.mean(sample, axis=0)

data_norm = data[["Alcalinity of ash", "Magnesium", "Proanthocyanins", "Hue"]]
a = mle_for_mean(data_norm.to_numpy())
a

array([19.51695, 99.58757,  1.58695,  0.95698])

**Вывод:** как и в предыдущем пункте, разница между обеими интервалами одного и того же параметра получилась незначительной. При этом доверительные интервалы для категорий `Alcalinity of ash` и `Magnesium` получились очень широкими (для `Alcalinity of ash` размер интервала примерно в 4 раза меньше оценки, а для `Magnesium` размер интервала и вовсе порядка оценки). Это связано с тем, что у колонок большая выборочная дисперсия.

_____________
## Задача 4.
На сегодняшний день возобновляемые источники энергии становятся все более востребованными. К таким источникам относятся, например, ветрогенераторы. Однако их мощность очень трудно прогнозировать. В частности, выработка энергии при помощи ветрогенератора сильно зависит от скорости ветра. Поэтому предсказание скорости ветра является очень важной задачей. Скорость ветра часто моделируют с помощью распределения Вейбулла, которое имеет плотность:
$$p_\theta(x) = \frac{kx^{k-1}}{\lambda^k} e^{-(x/\lambda)^k} I \{x \geq 0\},$$
где $\theta = (k, \lambda)$ &mdash; двумерный параметр. К сожалению, найти точную оценку максимального правдоподобия на $\theta$ не получится.  В данном задании нужно найти оценку максимального правдоподобия приближенно с помощью поиска по сетке.

За распределение Вейбулла отвечает класс `weibull_min` из модуля `scipy.stats`, которое задается так: `weibull_min(c=`$k$ `, scale=`$\lambda$ `)`.

**Выборка:** Создайте выборку по значеням среднесуточной скорости ветра на некоторой местности для нескольких лет (не менее трех). Данные доступны на вики. Можно выбрать любой файл.

*a).* Найдите оценку максимального правдоподобия параметра $\theta = (k, \lambda)$ с точностью $10^{-5}$ при помощи поиска по двумерной сетке.

Двумерную сетку можно создать с помощью функции `numpy.mgrid[from:to:step, from:to:step]`. Если попробовать сразу создать сетку с шагом $10^{-5},$ то может не хватить памяти. Поэтому найдите сначала максимум по сетке с большим шагом, затем сделайте сетку с маленьким шагом в окрестности найденной точки. При вычислении без циклов, возможно, придется создавать трехмерные объекты.

Функция `numpy.argmax` выдает не очень информативный индекс, поэтому пользуйтесь следующей функцией.

In [170]:
def cool_argmax(array):
    return np.unravel_index(np.argmax(array), array.shape)

In [177]:
X = sps.norm.rvs(size=[100, 100])

3146


(31, 46)

**Решение:**

Запишем функцию правдоподобия: 
$$L_X(\theta) = \frac{k^n}{\lambda^{nk}}\left(\prod\limits_{i = 1}^{n}X_i\right)^{k - 1} - e^{-\left(\frac{\sum\limits_{i = 1}^{n}X_i}{\lambda}\right)^k}.$$
Так как размер выборки может быть большим и могут встретиться значения, близкие по модулю к нулю, то чтобы избежать значений функции правдоподобия, близких к нулю, запишем логарифмическую функцию правдоподобия:
$$\mathcal{l}_X(\theta) = n\ln{k} - nk\ln{\lambda} + (k - 1)\sum\limits_{k = 1}^{n}\ln{X_i} -\left(\frac{\sum\limits_{i = 1}^{n}(X_i)^k}{\lambda^k}\right).$$
Будем искать по сетке максимум логарифмической функции правдоподобия.

In [295]:
data_weather = pd.read_csv('1.csv')
data_weather = data_weather[data_weather['Year'] >= 2010]
data_weather = data_weather['Mean'].dropna().to_numpy()
data_weather[:5]

array([3.2, 3.5, 1.9, 1.9, 3.9])

In [367]:
def grid_search_equation(
    func, from_lambda, to_lambda, from_k, to_k, step_lambda=0.,
    step_k=0.1, eps=1e-5, **kwargs
):
    '''
    Поиск по локальной сетке с уменьшающимся шагом 
    максимума функции f(lambda, k).
    
    Аргументы:
    func --- функция, максимум которой ищется;
    from_lambda, from_k --- минимильные значения параметров;
    to_lambda, to_k --- максимальные значения параметров;
    step_lambda, step_k --- начальные шаги сетки;
    eps --- точность, с которой надо найти корень.
    
    Возвращает:
    result_lambda, result_k --- найденные параметры, при которых f(lambda, k) максимально.
    '''
    
    while(step_k >= eps or step_lambda >= eps):
        # задаем сетку
        grid = np.mgrid[from_lambda:to_lambda:step_lambda, from_k:to_k:step_k]
        f_values = func(grid[0], grid[1], **kwargs)
        # находим текущее приближение
        max_lambda, max_k = cool_argmax(f_values)
        result_lambda = grid[0, max_lambda, 0]
        result_k = grid[1, 0, max_k]
        print(f'текущее значение функции правдоподобия: {f_values[max_lambda, max_k]}')
        # обновление параметров сетки
        from_lambda, to_lambda = result_lambda - step_lambda, result_lambda + step_lambda
        from_k, to_k = result_k - step_k, result_k + step_k
        step_lambda *= 0.1
        step_k *= 0.1
        
    return result_lambda, result_k

In [368]:
def loglikelihood_weibull(lambd, k, X):
    '''
    возвращает логарифмическую функцию правдоподобия распределения Вейбулла с параметрами (lambd, k) в точках выборки X
    '''
    k = np.array(k)
    n = len(X)
    if len(k.shape) == 2: # если k - двумерная матрица
        shapes = k.shape
        power = (X.reshape((n, 1)) ** (k.reshape((1, k.shape[0] * k.shape[1])))).reshape((n, shapes[0], shapes[1]))
    else: # если k - число
        power = X ** k
    return n * np.log(k) - n * k * np.log(lambd) + (k - 1) * np.sum(np.log(X)) - \
            (1 / np.power(lambd, k) * np.sum(power, axis=0))



def ll(lambd, k, X):
    return np.sum(sps.weibull_min(c=k, scale=lambd).logpdf(X))

In [373]:
lambd = 5.25
k = 2.11

result_lambda, result_k = grid_search_equation(func=loglikelihood_weibull, from_lambda=0.01, to_lambda=10,
                                               from_k=0.01, to_k=10, step_lambda=1, step_k=1, eps=1e-5, X=data_weather)
result_lambda, result_k

текущее значение функции правдоподобия: -5187.760617203618
текущее значение функции правдоподобия: -5176.121307396329
текущее значение функции правдоподобия: -5175.725524513101
текущее значение функции правдоподобия: -5175.717504476276
текущее значение функции правдоподобия: -5175.717488583822
текущее значение функции правдоподобия: -5175.717488064046


(5.257020000000001, 2.1128299999999998)

Нарисуйте график плотности с параметрами, соответствующим найденным ОМП, а так же нанесите на график гистограмму.

In [458]:
fig = go.Figure()
t = np.arange(np.min(data_weather) - 1, np.max(data_weather) + 1, 0.01)
fig.add_trace(go.Scatter(x=t, y=sps.weibull_min(c=result_k, scale=result_lambda).pdf(t), 
                         name='график плотности с параметрами, соответствующим найденным ОМП'))
fig.add_trace(go.Histogram(x=data_weather, nbinsx=20, histnorm='probability', name='гистограмма'))
fig.show()
pof.plot(fig, filename=f'hist_and_density.html', auto_open=False)

'hist_and_density.html'

*b).* Обозначим $\widehat{\theta} = \left(\widehat{\lambda}, \widehat{k}\right)$ &mdash; ОМП. Запишите уравнение правдоподобия, приравняв все частные производные в точке экстремума логарифмической функции правдоподобия к $0$. Используя одно из равенств, можно выразить $\widehat{\lambda}$ через значения $X_1, \dots, X_n, \widehat{k}$; подставив это выражение в другое равенство, получить уравнение на $\widehat{k}$. Решите это уравнение приближенно с помощью метода Ньютона и получите $\widehat{k}$, а значит, и $\widehat{\lambda}$.

**Решение:** вспомним, чему равна логарифмическая функция правдоподобия: $$\mathcal{l}_X(\theta) = n\ln{k} - nk\ln{\lambda} + (k - 1)\sum\limits_{k = 1}^{n}\ln{X_i} -\left(\frac{\sum\limits_{i = 1}^{n}(X_i)^k}{\lambda^k}\right).$$

Возьмем частные производные и приравняем их к нулю:
$$\frac{\partial \mathcal{l}_X(\theta)}{\partial \lambda} = -\frac{nk}{\lambda} + \frac{k \sum\limits_{i = 1}^{n}X_i^k}{\lambda^{k + 1}} = 0.$$
$$\frac{\partial \mathcal{l}_X(\theta)}{\partial k} = \frac{n}{k} - n\ln{\lambda} + \sum\limits_{i = 1}^{n}\ln{X_i} - \frac{\sum\limits_{i = 1}^{n}X_i^k(\ln{X_i} - \ln{\lambda})}{\lambda^k} = 0.$$
Из первого уравнения получаем: $$\widehat{\lambda} = \left(\frac{\sum\limits_{i = 1}^{n}X_i^k}{n}\right)^{\frac{1}{k}}.$$
Подставив полученную $$\widehat{\lambda}$$ во второе уравнение, получаем:
$$f(k) = \frac{n}{k} - \frac{n}{k}\ln{\sum\limits_{i = 1}^{n}X_i^k} + \frac{n\ln{n}}{k} + \sum\limits_{i = 1}^{n}\ln{X_i} - \frac{n\left(\sum\limits_{i = 1}^{n}X_i^k\left(\ln{X_i} - \frac{\ln{\sum\limits_{j = 1}^{n}X_j^k} - \ln n}{k}\right)\right)}{\sum\limits_{i = 1}^{n}X_i^k} = 0.$$

Найдем приближенное решение с помощью метода Ньютона. Численно произоводная ищется мега-неприятно, поэтому воспользуемся приближенной производной, которая совпадает с истинной с точностью до 2 порядка малости:
$$f_h'(x) = \frac{f(x + h) - f(x - h)}{2h}.$$

In [412]:
def f(k, X):
    n = len(X)
    sum_pow_k = np.sum(np.power(X, k))
    t = n / k * (1 - np.log(sum_pow_k)) + n * np.log(n) / k + np.sum(np.log(X))
    g = np.sum(np.power(X, k) * (np.log(X) - ((np.log(sum_pow_k) - np.log(n)) / k)))
    return t - (n * g / sum_pow_k)

f(2.11283, data_weather)

0.0024985656891658437

In [443]:
def approx_derivative(f, x, eps, **kwargs):
    return (f(x + eps, **kwargs) - f(x - eps, **kwargs)) / (2 * eps)

def newton_method(x0, num_iter, eps, f, **kwargs):
    x_prev = x0
    for i in range(num_iter):
        x = x_prev - f(x_prev, **kwargs) / approx_derivative(f, x_prev, eps, **kwargs)
        x_prev = x
    return x

k_newton = newton_method(1, 10000, 1e-5, f, X=data_weather)
k_newton

2.1128325144136535

In [444]:
def lambda_by_k(k, X):
    n = len(X)
    sum_pow_k = np.sum(np.power(X, k))
    return np.power((sum_pow_k / n), (1 / k))

lambda_newton = lambda_by_k(k_newton, data_weather)
print(f'k из метода Ньютона: {k_newton:.6f}')
print(f'lambda из метода Ньютона: {lambda_newton:.6f}')
print(f'k из ММП: {result_k:.6f}')
print(f'lambda из ММП: {result_lambda:.6f}')

k из метода Ньютона: 2.112833
lambda из метода Ньютона: 5.257018
k из ММП: 2.112830
lambda из ММП: 5.257020


**Вывод:** мы нашли оптимальное значение параметров распределения Вейбулла двумя способами: поиском по сетке максимума логарифмической функции правдоподобия и поиском решения уравнения правдоподобия с помощью метода Ньютона. Стоит заметить, что значения совпали с нужной точностью (с точностью примерно $3 \cdot 10^{-6}$). При этом у обоих методов проблема с функциями, где много экстремумов (хотя кажется, что поиск по сетке может найти нужный максимум с большей вероятностью), а также во втором методе возникает погрешность во-первых, от метода Ньютона, а во-вторых, от приближенного вычисления производной, в то время, как в первом методе для "хорошей" функции мы можем найти решение практически со сколь угодной точностью (эта точность как-то зависит от машинной точности).